In [ ]:
from graphics import *
import numpy as np
import time

In [ ]:
def unitVector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angleBetween(v1, v2, deg = True, debug = False):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::
    """
    if debug:
        print('Первый вектор:', v1)
        print('Второй вектор:', v2)
 
    v1_u = unitVector(v1)
    v2_u = unitVector(v2)
 
    if debug:
        print("v1 ", v1_u, " v2 ", v2_u)
    
    radians = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    result = radians
 
    if debug:
        print('радианы:', result)
    
    if deg:
        result = np.degrees([radians.real])[0]  # переводим в градусы
 
    return result


In [ ]:
class Model:
    def __init__(self, vertexes, edges, polygons, drawMode):
        self.vertexes = vertexes
        self.edges = edges
        self.polygons = polygons
        self.drawMode = drawMode

In [ ]:
dimond = Model([(0, 0.5, 0, 1), (0.5, 0, 0, 1), (0, 0, 0.5, 1), (-0.5, 0, 0, 1), (0, 0, -0.5, 1), (0, -0.5, 0, 1)],
               [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (2, 3), (3, 4), (4, 1), (5, 1), (5, 2), (5, 3), (5, 4)],
               [(0, 1, 2, "blue"), (0, 2, 3, "green"), (0, 3, 4, "pink"), (0, 4, 1, "orange"),
                (5, 2, 1, "yellow"), (5, 3, 2, "indigo"), (5, 4, 3, "purple"), (1, 4, 5, "white")],
               "model")

pyramid = Model([(0, 0, 0, 1), (-0.2, -0.5, -0.3, 1), (0.2, -0.5, -0.3, 1), (0, -0.5, 0.4, 1)],
                [(0, 1), (0, 2), (0, 3), (1, 2), (2, 3), (3, 1)],
                [(0, 1, 2), (0, 2, 3)],
                "both")

cube = Model([(0.5, 0.5, 0, 1), (0.5, -0.5, 0, 1), (-0.5, -0.5, 0, 1), (-0.5, 0.5, 0, 1), (0.5, 0.5, 1, 1),
              (0.5, -0.5, 1, 1), (-0.5, -0.5, 1, 1), (-0.5, 0.5, 1, 1)], #vertexes
             [(0, 1), (1, 2), (2, 3), (3, 0), (0, 4), (1, 5), (2, 6), (3, 7), (4, 5), (5, 6), (6, 7), (7, 4)], #edges
             [(0, 1, 2), (0, 2, 3), (6, 5, 4), (7, 6, 4), (2, 6, 7), (2, 5, 6), (6, 3, 2), (4, 6, 5), #polygons
              (0, 4, 5), (5, 1, 0),(0, 3, 7), (7, 4, 0), (6, 2, 1), (1, 5, 6)],
             "center")

                

In [ ]:
class App:
    def __init__(self, model):
        self.model = model
        self.resolution = (800, 800)
        self.window = GraphWin("Model", self.resolution[0], self.resolution[1], autoflush=True)
        self.window.setBackground('black')
        self.watching_point = (10, 10, 10)
        self.points = []
        self.projectedPoints = []
        self.rotatedPoints = []
        self.modelPoints = []
        self.readModel()
        self.angle = 0
        self.geometricCenter = (0, 0, 0)
        self.run()
        
    def getViewMatrix(self):
        theta = angleBetween(self.watching_point, (1, 0, 0), False)
        fi = angleBetween(self.watching_point, (0, 0, 1), False)
        ro = np.linalg.norm(self.watching_point)
        return [[-np.sin(theta), -np.cos(fi)*np.cos(theta), -np.sin(fi)*np.cos(theta), 0],
                [np.cos(theta), -np.cos(fi)*np.sin(theta), -np.sin(fi)*np.sin(theta), 0],
                [0, np.sin(fi), -np.cos(fi), 0],
                [0, 0, ro, 1]]

    def getRotationMatrix(self, angle, axis = "x"):
        return [[1, 0, 0, 0],
                [0, np.cos(np.radians(angle)), np.sin(np.radians(angle)), 0],
                [0, -np.sin(np.radians(angle)), np.cos(np.radians(angle)), 0],
                [0, 0, 0, 1]]
    
    def readModel(self):
        for vert in self.model.vertexes:
            vector = np.array(list(vert))
            self.modelPoints.append(vector)
            
    def rotatePoints(self):
        for point in self.modelPoints:
            result = np.dot(point, self.getRotationMatrix(self.angle))
            self.rotatedPoints.append(result)
    
    def transformPoints(self):
        for vert in self.rotatedPoints:
            vector = np.array(list(vert))
            result = np.dot(vector, self.getViewMatrix())
            self.points.append(result)
        self.calculateGeometricCenter()
    
    def projectPoints(self):
        ro = np.linalg.norm(self.watching_point)
        for point in self.points:
            x = ro/(2 * point[2]) * point[0]
            y = ro/(2 * point[2]) * point[1]
            self.projectedPoints.append((x, y)) 
    
    def normalizedPoint(self, point, width = 1, height = 1):
        x = (point[0] + width/2)/width
        y = (point[1] + height/2)/height
        return [x, y]
    
    def screenedPoint(self, point, width = 1, height = 1):
        x = point[0] * self.resolution[0]
        y = point[1] * self.resolution[1]
        return [x, y]
    
    def calculateGeometricCenter(self):
        xlist = [point[0] for point in self.points]
        ylist = [point[1] for point in self.points]
        zlist = [point[2] for point in self.points]
        self.geometricCenter = ((max(xlist)+min(xlist))/2, (max(ylist)+min(ylist))/2, (max(zlist)+min(zlist))/2) 
        
    def drawEdgesAndPolygons(self):
        
        if self.model.drawMode == "center":
            polygons = self.model.polygons
            for poly in polygons:
                #3D points
                p1 = self.points[poly[0]]
                p2 = self.points[poly[1]]
                p3 = self.points[poly[2]]
                
                a = p1[1] * (p2[2] - p3[2]) + p2[1] * (p3[2] - p1[2]) + p3[1] * (p1[2] - p2[2]);
                b = p1[2] * (p2[0] - p3[0]) + p2[2] * (p3[0] - p1[0]) + p3[2] * (p1[0] - p2[0]); 
                c = p1[0] * (p2[1] - p3[1]) + p2[0] * (p3[1] - p1[1]) + p3[0] * (p1[1] - p2[1]);
                d = -(p1[0] * (p2[1] * p3[2] - p3[1] * p2[2]) + p2[0] * (p3[1] * p1[2] - p1[1] * p3[2])
                      + p3[0] * (p1[1] * p2[2] - p2[1] * p1[2]));
                                
                p = self.geometricCenter
                I = a*p[0] + b*p[1] + c*p[2] + d
                n = np.array([a, b, c]) * np.sign(I)
                d = ((p1[0] + p2[0] + p3[0])/3, (p1[1] + p2[1] + p3[1])/3, (p1[2] + p2[2] + p3[2])/3) 
                
                angle = angleBetween(d, n)
                
                if angle < 90 and angle > 0:
                    #If we can see polygon, we're switching 3D points to projected 2D
                    self.drawPoly(poly)
        
        if self.model.drawMode == "model" or self.model.drawMode == "both":
            polygons = self.model.polygons
            for poly in polygons:
                #3D points
                p1 = self.points[poly[0]]
                p2 = self.points[poly[1]]
                p3 = self.points[poly[2]]

                v1 = (p2[0] - p1[0], p2[1] - p1[1], p2[2] - p1[2]) 
                v2 = (p3[0] - p1[0], p3[1] - p1[1], p3[2] - p1[2]) 

                n = (v1[1]*v2[2] - v1[2]*v2[1], v1[2]*v2[0] - v1[0]*v2[2], v1[0]*v2[1] - v1[1]*v2[0])

                if (-p1[0]*n[0] + -p1[1]*n[1] + -p1[2]*n[2] <= 0):
                    #If we can see polygon, we're switching 3D points to projected 2D
                    self.drawPoly(poly)
                    

        if self.model.drawMode == "carcass" or self.model.drawMode == "both":            
            edges = self.model.edges
            for edge in edges:
                point1 = self.projectedPoints[edge[0]]
                point2 = self.projectedPoints[edge[1]]

                point1 = self.normalizedPoint(point1, 1, 1)
                point2 = self.normalizedPoint(point2, 1, 1)

                point1 = self.screenedPoint(point1)
                point2 = self.screenedPoint(point2)

                line = Line(Point(point1[0], point1[1]), Point(point2[0], point2[1]))
                line.setFill("blue")
                line.draw(self.window)
  
    def drawPoly(self, poly):
        p1 = self.projectedPoints[poly[0]]
        p2 = self.projectedPoints[poly[1]]
        p3 = self.projectedPoints[poly[2]]

        p1 = self.normalizedPoint(p1, 1, 1)
        p2 = self.normalizedPoint(p2, 1, 1)
        p3 = self.normalizedPoint(p3, 1, 1)

        p1 = self.screenedPoint(p1)
        p2 = self.screenedPoint(p2)
        p3 = self.screenedPoint(p3)

        point1 = Point(p1[0], p1[1])
        point2 = Point(p2[0], p2[1])
        point3 = Point(p3[0], p3[1])

        triangle = Polygon([point1, point2, point3])
                    
        if len(poly) > 3:
            triangle.setFill(poly[3])
        else:
            triangle.setFill('blue')
    
        triangle.setOutline('gray')
        triangle.setWidth(2)  # width of boundary line
        triangle.draw(self.window)
        
    def draw(self):
        self.clear()
        self.drawModel()
    
    def clear(self):
        for item in self.window.items[:]:
            item.undraw()
        self.points = []
        self.projectedPoints = []
        self.rotatedPoints = []
        self.window.update()
    
    def drawModel(self):
        self.rotatePoints()
        self.transformPoints()
        self.projectPoints()
        self.drawEdgesAndPolygons()

    def run(self):
        while True:
            self.update()
        
    def update(self):
        self.angle += 5
        self.draw()
        time.sleep(0.1)
   

In [ ]:

a = App(cube)
